In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import train_test_split, KFold

cudnn.benchmark = True
plt.ion()   # interactive mode
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

test_size = 0.25

data_train = np.load('data_train.npy').reshape((300, 300, 9032))
data_train = np.repeat(np.expand_dims(data_train, axis = 2), 3, axis=2).T
labels_train = np.load('t_train.npy')

tensor_data = torch.Tensor(data_train) # transform to torch tensor
tensor_labels = torch.Tensor(labels_train)

dataset = TensorDataset(tensor_data, tensor_labels) # create your datset

train_indices, test_indices = train_test_split(range(len(dataset)), test_size=test_size, shuffle=True)

train_subset = Subset(dataset, train_indices)
test_subset = Subset(dataset, test_indices)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                labels = labels.type(torch.LongTensor) 
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [11]:
def test_model(model, criterion, optimizer, scheduler):
    since = time.time()

    # Each epoch has a training and validation phase
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in dataloader_test:
        labels = labels.type(torch.LongTensor) 
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        
        
        

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    loss = running_loss / dataset_test_size
    acc = running_corrects.double() / dataset_test_size

    print(f'Loss: {loss:.4f} Acc: {acc:.4f}')

    print()

    time_elapsed = time.time() - since
    print(f'Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model

In [5]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [6]:
kf = KFold(n_splits=10)
for train_index, val_index in kf.split(train_subset):
    train_subset_fold = Subset(train_subset, train_index)
    val_subset_fold = Subset(train_subset, val_index)
    dataset = {x: train_subset_fold if x == 'train' else val_subset_fold
                      for x in ['train', 'val']}

    dataloaders = {x: DataLoader(dataset[x], batch_size = 4, shuffle = True, num_workers = 4) for x in ['train', 'val']} # create your dataloader

    dataset_sizes = {x: len(dataset[x]) for x in ['train', 'val']}

    model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                           num_epochs=10)

Epoch 0/9
----------


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 0.8320 Acc: 0.7469
val Loss: 0.5955 Acc: 0.8260

Epoch 1/9
----------
train Loss: 0.4269 Acc: 0.8811
val Loss: 0.3523 Acc: 0.9174

Epoch 2/9
----------
train Loss: 0.3160 Acc: 0.9122
val Loss: 0.3885 Acc: 0.9159

Epoch 3/9
----------
train Loss: 0.2510 Acc: 0.9286
val Loss: 0.3526 Acc: 0.9351

Epoch 4/9
----------
train Loss: 0.1948 Acc: 0.9449
val Loss: 0.3007 Acc: 0.9381

Epoch 5/9
----------
train Loss: 0.1549 Acc: 0.9532
val Loss: 0.3340 Acc: 0.9322

Epoch 6/9
----------
train Loss: 0.1147 Acc: 0.9672
val Loss: 0.2890 Acc: 0.9425

Epoch 7/9
----------
train Loss: 0.0601 Acc: 0.9831
val Loss: 0.2797 Acc: 0.9454

Epoch 8/9
----------
train Loss: 0.0477 Acc: 0.9872
val Loss: 0.2739 Acc: 0.9469

Epoch 9/9
----------
train Loss: 0.0473 Acc: 0.9902
val Loss: 0.2826 Acc: 0.9469

Training complete in 5m 53s
Best val Acc: 0.946903
Epoch 0/9
----------
train Loss: 0.0717 Acc: 0.9838
val Loss: 0.0188 Acc: 0.9956

Epoch 1/9
----------
train Loss: 0.0597 Acc: 0.9870
val Loss: 0.0192

val Loss: 0.0191 Acc: 0.9985

Epoch 5/9
----------
train Loss: 0.0575 Acc: 0.9880
val Loss: 0.0204 Acc: 0.9970

Epoch 6/9
----------
train Loss: 0.0592 Acc: 0.9879
val Loss: 0.0182 Acc: 0.9970

Epoch 7/9
----------
train Loss: 0.0658 Acc: 0.9846
val Loss: 0.0181 Acc: 0.9985

Epoch 8/9
----------
train Loss: 0.0606 Acc: 0.9864
val Loss: 0.0214 Acc: 0.9970

Epoch 9/9
----------
train Loss: 0.0607 Acc: 0.9875
val Loss: 0.0188 Acc: 0.9970

Training complete in 5m 36s
Best val Acc: 0.998523


In [12]:
dataloader_test = DataLoader(test_subset, batch_size = 4, shuffle = True, num_workers = 4)
dataset_test_size = len(test_subset)
test_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler)

Loss: 0.2684 Acc: 0.9429

Testing complete in 0m 12s


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for img in 